In [3]:
# 反思
import os

from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.messages import SystemMessage, HumanMessage

if not os.getenv('DEEPSEEK_SK'):
    raise ValueError('不存在deepseek sk')

llm = ChatOpenAI(base_url='https://api.deepseek.com', api_key=os.getenv('DEEPSEEK_SK'), model='deepseek-chat')

def run_reflection_loop():
    """
        演示多步AI循环反思
    """
    # 核心任务
    task_prompt = """
    你的任务是创建一个名为 `calculate_factorial` 的 Python 函数。
    该函数需满足以下要求：
    1. 只接受一个整数参数 n。
    2. 计算其阶乘（n!）。
    3. 包含清晰的 docstring，说明函数功能。
    4. 处理边界情况：0 的阶乘为 1。
    5. 处理无效输入：若输入为负数则抛出 ValueError。
    """
    max_iterations = 3
    # 当前代码
    current_code = ''
    # 构建历史对话
    history_message = [HumanMessage(content=task_prompt)]
    
    for i in range(max_iterations):
        print('\n' + '='*10 + f'第{i}次反思循环' + '='*10)
        
        if i == 0:
            print('第一步：初始化代码')
            response = llm.invoke(history_message)
            current_code = response.content
        else:
            print('第一步：根据批判优化代码')
            history_message.append(HumanMessage(content='请根据批判意见优化代码'))
            response = llm.invoke(history_message)
            current_code = response.content
            
        print(f'生成第{i}版代码')
        history_message.append(response)
        
        # 反思
        print('第二步：对代码进行反思')
        reflector_prompt = [
            SystemMessage(content='''
            你是一名资深软件工程师，精通 Python。
            你的职责是对提供的 Python 代码进行细致代码审查。
            请根据原始任务要求，严格评估代码。
            检查是否有 bug、风格问题、遗漏边界情况及其他可改进之处。
            若代码完美且满足所有要求，仅回复 'CODE_IS_PERFECT'。
            否则，请以项目符号列表形式给出批判意见。
            '''),
            HumanMessage(content=f'原始任务:{task_prompt}, 待评审代码:{current_code}')
        ]
        
        critique_response = llm.invoke(reflector_prompt)
        critique_content = critique_response.content
        
        if critique_content == 'CODE_IS_PERFECT':
            print('='*10 + '代码达到要求' + '='*10)
            break
        else:
            print(f'第{i}次批判内容: {critique_content}')
            history_message.append(HumanMessage(content=f'上次批判意见: {critique_content}'))
    
    print('='*10 + '最终结果: ' + '='*10 + '\n')
    print(current_code)
    
if __name__ == '__main__':
    run_reflection_loop()
            



==========第0次反思循环==========
第一步：初始化代码
生成第0版代码
第二步：对代码进行反思
==========代码达到要求==========
==========最终结果: ==========

```python
def calculate_factorial(n):
    """
    计算非负整数 n 的阶乘 (n!)。
    
    阶乘定义为从 1 到 n 的所有正整数的乘积。特别地，0 的阶乘定义为 1。
    
    参数:
        n (int): 要计算阶乘的非负整数
        
    返回:
        int: n 的阶乘结果
        
    异常:
        ValueError: 如果 n 为负数
    """
    if n < 0:
        raise ValueError("阶乘只能计算非负整数，输入不能为负数")
    
    if n == 0:
        return 1
    
    result = 1
    for i in range(1, n + 1):
        result *= i
    
    return result
```

这个函数实现了所有要求的功能：

1. **参数处理**：只接受一个整数参数 `n`
2. **阶乘计算**：使用循环计算从 1 到 n 的乘积
3. **完整文档**：包含清晰的 docstring，说明函数功能、参数、返回值和异常
4. **边界情况**：正确处理 0 的阶乘为 1
5. **错误处理**：对负数输入抛出 ValueError 异常

**使用示例：**
```python
print(calculate_factorial(5))  # 输出: 120
print(calculate_factorial(0))  # 输出: 1
print(calculate_factorial(1))  # 输出: 1

# 负数会抛出异常
# calculate_factorial(-1)  # ValueError: 阶乘只能计算非负整数，输入不能为负数
```
